In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import glob
import json
import numpy as np
import os
import shutil


import torch
from tqdm import tqdm
from pycocotools.coco import COCO  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb
import clip

from vis_prior.utils import *

/opt/conda/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
unfiltered_data_folder = "/media/data/dad/cnet/experiments/coco10novel/mix_n2000_o1_s1_p640_pfa_csl30"
unfiltered_annotation_path = os.path.join(unfiltered_data_folder, "annotation.json")

In [4]:
with open(unfiltered_annotation_path, "r") as f:
    unfiltered_annotation = json.load(f)
coco = COCO(unfiltered_annotation_path)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
device = "cuda"
model, preprocess = clip.load("ViT-L/14", device=device)

In [6]:
prompts = [cat['name'] for cat in unfiltered_annotation['categories']]

In [7]:
prompts

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'boat',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'bottle',
 'chair',
 'couch',
 'potted plant',
 'dining table',
 'tv']

In [8]:
from PIL import Image, ImageDraw
for img_obj in tqdm(unfiltered_annotation['images']):
    image_id = img_obj["id"]
    img_path = os.path.join(unfiltered_data_folder, 'images', img_obj['file_name'])
    img = Image.open(img_path)
    
    ann_ids = coco.getAnnIds(imgIds=image_id)
    imgdraw = ImageDraw.Draw(img)  
    cat_ids = set()
    for ann_id in ann_ids:
        ann = coco.loadAnns([ann_id])[0]
        x, y, w, h = ann['bbox']
        imgdraw.rectangle((x, y, x+w, y+h), fill="black")
        cat_ids.add(ann['category_id'])
    
    
    cat_objs = coco.loadCats(cat_ids)
    #cat_name = cat_obj["name"]
    cat_names = [cat_obj["name"] for cat_obj in cat_objs] # only detect for categories assigned
    
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(cat_names).to(device)
    
    #import matplotlib.pyplot as plt
    #plt.imshow(img)
    #print([cat_name, "a " + cat_name, "picture of a " + cat_name])
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        
    #print(cat_prompts)
    #print(logits_per_image)
    
    for i, cat_name in enumerate(cat_names):
        img_obj["csl_%s"%cat_name] = float(logits_per_image[0][i]) # CLIP Similarity Score VIT-L14 (csl)    
    #ann["csl_a"] = float(probs[1]) # a: "a xxxx"    
    #ann["csl_p"] = float(probs[2]) # p: "a picture of xxx"

100%|█████████████████████████████████████████████████████████████████████████| 729/729 [00:45<00:00, 16.05it/s]


In [9]:
print(len(unfiltered_annotation['images']))
print(unfiltered_annotation['images'][222])
print(unfiltered_annotation['images'][223])
print(unfiltered_annotation['images'][224])
print(unfiltered_annotation['images'][225])

729
{'file_name': '000000570335.jpg', 'height': 640, 'width': 640, 'id': 570335, 'prompt': 'cat', 'csl_cat': 20.890625}
{'file_name': '000000570341.jpg', 'height': 640, 'width': 640, 'id': 570341, 'prompt': 'motorcycle', 'csl_motorcycle': 11.71875}
{'file_name': '000000570343.jpg', 'height': 640, 'width': 640, 'id': 570343, 'prompt': 'train', 'csl_train': 20.734375}
{'file_name': '000000570345.jpg', 'height': 640, 'width': 640, 'id': 570345, 'prompt': 'airplane', 'csl_airplane': 19.046875}


In [10]:
with open(unfiltered_annotation_path, "w+") as f:
    json.dump(unfiltered_annotation, f)